In [ ]:
# Imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
# Function and constant definitions

FINGERS = [
    'left-5-x',
    'left-5-y',
    'left-5-z',
    'left-4-x',
    'left-4-y',
    'left-4-z',
    'left-3-x',
    'left-3-y',
    'left-3-z',
    'left-2-x',
    'left-2-y',
    'left-2-z',
    'left-1-x',
    'left-1-y',
    'left-1-z',
    'right-1-x',
    'right-1-y',
    'right-1-z',
    'right-2-x',
    'right-2-y',
    'right-2-z',
    'right-3-x',
    'right-3-y',
    'right-3-z',
    'right-4-x',
    'right-4-y',
    'right-4-z',
    'right-5-x',
    'right-5-y',
    'right-5-z',    
]

def make_batches(X, y, window_size=10):
    ends = np.array(range(window_size, len(y) - 1))
#     ends = np.nonzero(y != 'gesture0255')[0] + 1
    starts = ends - window_size
    batched_X = np.empty((ends.shape[0], window_size, X.shape[1]))
    batched_y = np.empty((ends.shape[0],), dtype='object')
    for i in range(batched_y.shape[0]):
        batched_X[i] = X[starts[i]:ends[i]]
        batched_y[i] = y[ends[i]]
    return batched_X, batched_y

def gestures_and_indices(y):
    labels = sorted(np.unique(y))
    g2i_dict = {g:i for i, g in enumerate(labels)}
    i2g_dict = {i:g for i, g in enumerate(labels)}
    def g2i(g):
        return np.array([g2i_dict[gi] for gi in g])
    def i2g(i):
        return np.array([i2g_dict[ii] for ii in i])
    return g2i, i2g

def one_hot_and_back(y_all):
    return (
        lambda y: tf.one_hot(y, len(np.unique(y_all))),
        lambda onehot: tf.argmax(one_hot, axis=1)
    )

def conf_mat(model, X, y):
    y_pred = np.argmax(model.predict(X), axis=1)
    y_true = y

    confusion_mtx = tf.math.confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(confusion_mtx, annot=True, fmt='g')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')


In [ ]:
df = pd.read_csv(
    '../gesture_data/train/2022-08-18T21:24:24.314934.csv',
    names=['datetime', 'gesture'] + FINGERS,
    parse_dates=[1]
)
df.datetime = df.datetime.apply(pd.Timestamp)
X, y = make_batches(    
    df.drop(['datetime', 'gesture'], axis=1).to_numpy(), 
    df['gesture'].to_numpy()
)
# Get functions to convert between gestures and indices
g2i, i2g = gestures_and_indices(y)
y = g2i(y)
# Get functions to convert between indices and one hot encodings
i2ohe, ohe2i = one_hot_and_back(y)

total = len(y)
n_unique = len(np.unique(y))
class_weight = {
    class_: (1/weight * total/n_unique) for class_, weight in zip(*np.unique(y, return_counts=True))
}
X_train, X_valid = X[total//4:], X[:total//4]
y_train, y_valid = y[total//4:], y[:total//4]

In [ ]:
inputs = layers.Input(shape=X.shape[1:])

normalizer = layers.Normalization(axis=-1)
normalizer.adapt(X)
x = normalizer(inputs)

x = layers.Flatten()(x)
x = layers.Dense(units=256)(x)

outputs = layers.Dense(len(np.unique(y)), activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy('acc')],
)

history = model.fit(
    X, 
    y,
    batch_size=32, 
    epochs=30,
    validation_data=(X_valid, y_valid),
#     class_weight=class_weight,
)


In [ ]:
conf_mat(model, X_valid, y_valid)
plt.title('Validation set')
plt.show()

conf_mat(model, X_train, y_train)
plt.title('Training set')
plt.show

---